In [15]:
# constructs level2 summary file from level 1b file using manual ifcb data
# requires: level_1b.csv, resolved_manual.csv, volumes.csv, geographic_subset.csv
import pandas as pd
import numpy as np

In [16]:
# read in level 1b file
columns = ['permalink', 'data_provider_category_HumanObservation', 
               'higherClassification_group', 'scientificName_HumanObservation',
               'scientificNameID_HumanObservation', 'Biovolume']
# initialize data frame from input
samples = pd.read_csv("../auto_join/level_1b_manual.csv", usecols=columns)

In [17]:
# read in taxonomic information
man_taxon_info = pd.read_csv('../auto_join/resolved_manual_matched_matchIDs_LOOKUPsorted.csv', 
                              usecols=['name', 'resolved_names', 'alt_datasource'])
# merge to get taxa data
samples = pd.merge(samples, man_taxon_info, how='left', left_on='data_provider_category_HumanObservation', right_on='name')
samples.rename(columns={'higherClassification_group':'higherClassification_group_manual'}, inplace=True)

In [18]:
# separate out roi id from permalink
samples['roi'] = samples['permalink']
samples.roi = samples.roi.str.slice(68, 74)
# gets rid of leading zeros
samples.roi = samples.roi.str.lstrip("0")
# cut permalink to just be permalink of sample
samples.permalink = samples.permalink.str.slice(0, 67)

In [19]:
# read in volume data
volumes = pd.read_csv("../volume/volumes.csv")
# merge with samples
samples = pd.merge(samples, volumes, how='left', on='permalink')

In [20]:
# read in geolocation data
geo_data = pd.read_csv("geographic_subset.csv", usecols=['gps_furuno_latitude', 'gps_furuno_longitude', 
                                                         'date', 'pid'])
samples = pd.merge(samples, geo_data, how='left', left_on='permalink', right_on='pid')

In [21]:
# append .html to permalink
samples['permalink'] = samples['permalink'] + '.html'

In [22]:
# make level_2 summary file from manual data 
level_2 = samples
level_2 = level_2.groupby(['permalink', 'data_provider_category_HumanObservation']).agg(
    {
        'scientificName_HumanObservation': 'first',
        'scientificNameID_HumanObservation': 'first',
        'higherClassification_group_manual': 'first',
        'roi': 'count',
        'Biovolume': 'sum',
        'volume_imaged': 'first',
        'date': 'first',
        'gps_furuno_latitude': 'first',
        'gps_furuno_longitude': 'first'
    }
).reset_index()
# rename roi to be abundance
level_2.rename(columns={'roi': 'Abundance'}, inplace=True)

In [23]:
# save output to csv file
level_2.to_csv('level_2_manual.csv', index=None, header=True)